In [ ]:
# Load dependencies
Sys.setenv(LANGUAGE = "en") # set language to "ja" if you prefer
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(DiffBind))
suppressPackageStartupMessages(library(grid))

getwd()
sessionInfo()

In [ ]:
save_figure <- function(p,filename, width = unit(7,"cm"), height = unit(7,"cm")) {
    # Save to SVG
    svg(filename, width = width, height = height)
    replayPlot(p)
    dev.off()
}

In [ ]:
load_sample_metadata <- function(sample_sheet_path = '../CUT&Tag_sample_information.xlsx'){
    df = readxl::read_excel(sample_sheet_path) %>%
        dplyr::rename("Antibody"="Target Antibody","SampleID"="File Name","Replicate"="Rep") %>%
        dplyr::mutate(
            DMSO = stringr::str_detect(Treatment, "DMSO"),
            UNC0379 = stringr::str_detect(Treatment, "UNC0379"),
            DS1001b = stringr::str_detect(Treatment, "DS-1001b"),
            SampleID = stringr::str_replace_all(SampleID, "Combi|Comb", "combo"),
            Peaks = stringr::str_c("../macs3/",SampleID,"_peaks.NarrowPeak"),
            bamReads = stringr::str_c("../bowtie2/",SampleID,".sort.bam"),
            bamControl = stringr::str_c("../bowtie2/",str_extract(SampleID, "^[^_]+"),"_ctrl_",Replicate,".sort.bam"),
            #PeakCaller = 'narrow',
            Factor = Antibody,
            Condition = if_else(UNC0379, "UNC0379", "no_UNC0379"),
            Treatment = if_else(DS1001b, "DS1001b", "no_DS1001b")
        ) %>%
        dplyr::select(-"Sample No.",-"Date of Prep",-"Cell Line") %>%
        dplyr::filter(Antibody != "IgG")
    return(df)
}
sample_metadata = load_sample_metadata()

In [ ]:
sample_metadata

# Joint analysis. 
This is good for the limited use cases where we want to directly compare H3K27ac to H4K20Me1.

In [ ]:
# Annoying, but DiffBind only accepts specific keywords in its experimental design. Therefore,
# Factor <- Antibody
# Condition <- UNC0379
# Treatment <- DS1001-b
dba_all = DiffBind::dba(sampleSheet=sample_metadata %>% correct_swapped_samples,
                        peakCaller='narrow') %>% suppressWarnings

In [ ]:
plot(dba_all)
p <- recordPlot()
save_figure(p,"out/heatmap_all.svg")


# Histone modification specific analyses

In [ ]:
get_histone_specific_dba <- function(metadata_table,histone){
    df <- metadata_table %>%
        dplyr::filter(Antibody == histone)
    da <- DiffBind::dba(sampleSheet=df,peakCaller='narrow') %>% suppressWarnings
    return(da)
}
correct_swapped_samples <- function(table){
    # DMSO_H3K27ac_1 and DMSO_H4K20Me1_1 are suspected sample swaps. Swap them back.
    table <- table %>%
        mutate(across(c(Peaks, bamReads), function(x) {
            x %>%
                str_replace_all("DMSO_H3K27ac_1", "__TMP__") %>%
                str_replace_all("DMSO_H4K20me1_1", "DMSO_H3K27ac_1") %>%
                str_replace_all("__TMP__", "DMSO_H4K20me1_1")
      }))
    return(table)
}

## H3K27ac

In [ ]:
dba_h3k27ac = get_histone_specific_dba(sample_metadata %>% correct_swapped_samples,'H3K27ac')
#dba_h3k27ac = get_histone_specific_dba(sample_metadata,'H3K27ac')
dba_h3k27ac

In [ ]:
plot(dba_h3k27ac)
p <- recordPlot()
save_figure(p,"out/heatmap_h3k27ac_peaks.svg")

In [ ]:
# This takes a *very* long time and uses a ton of memory.
dba_h3k27ac <- dba.count(dba_h3k27ac,summits=FALSE,minOverlap=2,bUseSummarizeOverlaps=TRUE,bParallel=TRUE)

In [ ]:
plot(dba_h3k27ac)
p <- recordPlot()
save_figure(p,"out/heatmap_h3k27ac_counts.svg")

In [ ]:
dba_h3k27ac <- dba.normalize(dba_h3k27ac)

In [ ]:
# Condition <- UNC0379
# Treatment <- DS1001-b
dba_h3k27ac <- dba.contrast(dba_h3k27ac,design="~Treatment + Condition + Replicate",
                           reorderMeta = list(Treatment="DS1001b",Condition="UNC0379"))

In [ ]:
# DiffBind bug stemming from use of NCBI chromosome names. Solution from
# https://support.bioconductor.org/p/9138617/#9148413
pv.countGreylistEdited <- function (bamfile, pv, ktype) {
  #gl <- new("GreyList", karyotype = ktype[pv$chrmap, ]) #previous line
  #edit to restrict to just chromosomes included in the ktype object
  gl <- new("GreyList", karyotype = ktype[intersect(pv$chrmap, names(ktype)), ])
  gl <- GreyListChIP::countReads(gl, bamfile)
  return(gl)
}

environment(pv.countGreylistEdited) <- asNamespace('DiffBind')
assignInNamespace("pv.countGreylist", pv.countGreylistEdited, ns = "DiffBind")

In [ ]:
dba_h3k27ac <- dba.analyze(dba_h3k27ac)
# Show contrasts
dba.show(dba_h3k27ac,bContrast=T)

In [ ]:
plot_save_differential_peaks <- function(dbo,name,contrast){
    # write differential genes
    res <- dba.report(dbo,contrast=contrast,bFlip=T)
    out <- as.data.frame(res)
    write.table(out, file=paste0("out/",name,"_differential_peaks.tsv"), sep="\t", quote=F, row.names=F)
    # MA plot
    dba.plotMA(dbo,contrast=contrast,bFlip=T)
    p <- recordPlot()
    save_figure(p,paste0("out/",name,"_MAplot.svg"))
    # Volcano plot
    dba.plotVolcano(dbo,contrast=contrast,bFlip=T)
    p <- recordPlot()
    save_figure(p,paste0("out/",name,"_volcanoplot.svg"))
    return()
}

In [ ]:
plot_save_differential_peaks(dba_h3k27ac,"h3k27ac_DS-1001b",1)

In [ ]:
plot_save_differential_peaks(dba_h3k27ac,"h3k27ac_UNC0379",2)

### H4K20me1

In [ ]:
dba_h4k20me1 = get_histone_specific_dba(sample_metadata %>% correct_swapped_samples,'H4K20me1')
#dba_h4k20me1 = get_histone_specific_dba(sample_metadata,'H4K20me1')
dba_h4k20me1

In [ ]:
plot(dba_h4k20me1)
p <- recordPlot()
save_figure(p,"out/heatmap_h4k20me1_peaks.svg")

In [ ]:
# This takes a *very* long time and uses a ton of memory.
dba_h4k20me1 <- dba.count(dba_h4k20me1,summits=FALSE,minOverlap=2,bUseSummarizeOverlaps=TRUE,bParallel=TRUE)
plot(dba_h4k20me1)
p <- recordPlot()
save_figure(p,"out/heatmap_h4k20me1_counts.svg")

In [ ]:
dba_h4k20me1 <- dba.normalize(dba_h4k20me1)

In [ ]:
dba_h4k20me1 <- dba.contrast(dba_h4k20me1,design="~Treatment + Condition + Replicate",
                           reorderMeta = list(Treatment="DS1001b",Condition="UNC0379"))

In [ ]:
dba_h4k20me1 <- dba.analyze(dba_h4k20me1)
# Show contrasts
dba.show(dba_h4k20me1,bContrast=T)

In [ ]:
plot_save_differential_peaks(dba_h4k20me1,"h4k20me1_DS-1001b",1)

In [ ]:
plot_save_differential_peaks(dba_h4k20me1,"h4k20me1_UNC0379",2)